In [1]:
import gym
from torch.nn import functional as F
import torch
import torch.nn as nn

In [13]:
class Agent(nn.Module):
    def __init__(self):
        super(Agent,self).__init__()
        
        self.dense1= nn.Linear(4,15)
        self.dense2= nn.Linear(15,15)
        
        self.dense3= nn.Linear(15,2)
        
        
    def forward(self,x):
        
        out = self.dense1(x)
        out = F.relu(out)
        
        out = F.dropout(out)
        
        out = self.dense2(out)
        out = F.relu(out)
        
        out = self.dense3(out)
        out = F.softmax(out,dim=-1)
        
        
        return out

In [22]:
net = Agent()

In [23]:
net

Agent(
  (dense1): Linear(in_features=4, out_features=15, bias=True)
  (dense2): Linear(in_features=15, out_features=15, bias=True)
  (dense3): Linear(in_features=15, out_features=2, bias=True)
)

In [24]:
env = gym.make('CartPole-v0')


In [25]:
import numpy as np

def sample_action(action_probs):
    p = action_probs
    p = p.ravel()
    
    cum_p= np.cumsum(p)
    
    r=np.random.random()
    
    for i in range(len(p)):
        
        if r<cum_p[i]:
            return i
        
        
        
def discount(rewards):
    gamma =0.96
    
    rewards = np.array(rewards)
    discounted_rewards = np.zeros_like(rewards)
    
    for i in range(len(rewards)):
        cumsum=0
        for j in range(i,len(rewards)):
            cumsum+= gamma**(j-i) * rewards[j]
        
        discounted_rewards[i]=cumsum
        
    mean=discounted_rewards.mean()
    std=discounted_rewards.std()
    
    discounted_rewards-=mean
    discounted_rewards/=std
        
    return discounted_rewards
    
        

optimiser=torch.optim.Adam(net.parameters(),lr=0.01)
count=0
for i in range(1000):

    states,actions,rewards=[],[],[]
    state=env.reset()
    done=False
    steps=0

    while not done:
    
        states.append(state)
        state = torch.from_numpy(state).unsqueeze(0).float()
        action_probs = net(state).data.numpy().squeeze(0)
        

        action = sample_action(action_probs)
        actions.append(action)

        nextstate,reward,done,_ = env.step(action)
        rewards.append(reward)

        state = nextstate
        steps+=1
    
    states=np.array(states)
    actions=np.array(actions)
    rewards=np.array(rewards)
    
    states = torch.from_numpy(states).float()
    actions = torch.from_numpy(actions).long()
    
    action_prob_log = net(states)
    
    discounted_rewards = discount(rewards)
    discounted_rewards=torch.from_numpy(discounted_rewards).float()
    
    neg_logloss = F.cross_entropy(action_prob_log,actions,reduce=False)
    
    loss = torch.mean(neg_logloss * discounted_rewards)
    optimiser.zero_grad()
    
    loss.backward()
    
    optimiser.step()
    
    if rewards.sum()==200:
        count+=1
        if count==5:
            break
    
    if i%20==0:
        print(rewards.sum())
    
    
    
        
    
    
    

/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


25.0
13.0
19.0
21.0
10.0
11.0
11.0
10.0
10.0
12.0
13.0
19.0
14.0
21.0
15.0
10.0
13.0
15.0
13.0
9.0
65.0
88.0
88.0


In [46]:
F.cross_entropy(torch.randn(10,3).float(),torch.randint(size=(10,),high=3).long())

tensor(1.1033)

In [34]:
len(neg_logloss)==steps

True

In [27]:
len(actions)==len(action_prob_log)

True

In [39]:
discount(rewards)

array([4.99896154, 4.99870193, 4.99837741, 4.99797176, 4.9974647 ,
       4.99683087, 4.99603859, 4.99504824, 4.9938103 , 4.99226287,
       4.99032859, 4.98791074, 4.98488843, 4.98111053, 4.97638817,
       4.97048521, 4.96310651, 4.95388314, 4.94235392, 4.92794241,
       4.90992801, 4.88741001, 4.85926251, 4.82407814, 4.78009767,
       4.72512209, 4.65640262, 4.57050327, 4.46312909, 4.32891136,
       4.1611392 , 3.951424  , 3.68928   , 3.3616    , 2.952     ,
       2.44      , 1.8       , 1.        ])

In [41]:
loss.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [42]:
neg_logloss

tensor([0.5991, 0.7990, 0.5991, 0.5973, 0.5927, 0.5855, 0.8212, 0.8133, 0.8049,
        0.8003, 0.7983, 0.7955, 0.6008, 0.7949, 0.6010, 0.7946, 0.7945, 0.6027,
        0.6013, 0.7940, 0.6012, 0.7938, 0.6011, 0.7936, 0.6010, 0.7939, 0.6008,
        0.6013, 0.6002, 0.7978, 0.7954, 0.6003, 0.7951, 0.7959, 0.7954, 0.7951,
        0.7965, 0.5986])

In [44]:
net(torch.from_numpy(states).float())

tensor([[0.4011, 0.5989],
        [0.3992, 0.6008],
        [0.4011, 0.5989],
        [0.3991, 0.6009],
        [0.3940, 0.6060],
        [0.3859, 0.6141],
        [0.3792, 0.6208],
        [0.3863, 0.6137],
        [0.3939, 0.6061],
        [0.3980, 0.6020],
        [0.3999, 0.6001],
        [0.4024, 0.5976],
        [0.4029, 0.5971],
        [0.4030, 0.5970],
        [0.4032, 0.5968],
        [0.4032, 0.5968],
        [0.4033, 0.5967],
        [0.4050, 0.5950],
        [0.4035, 0.5965],
        [0.4038, 0.5962],
        [0.4034, 0.5966],
        [0.4040, 0.5960],
        [0.4033, 0.5967],
        [0.4041, 0.5959],
        [0.4032, 0.5968],
        [0.4039, 0.5961],
        [0.4029, 0.5971],
        [0.4035, 0.5965],
        [0.4023, 0.5977],
        [0.4003, 0.5997],
        [0.4025, 0.5975],
        [0.4024, 0.5976],
        [0.4028, 0.5972],
        [0.4020, 0.5980],
        [0.4025, 0.5975],
        [0.4028, 0.5972],
        [0.4015, 0.5985],
        [0.4005, 0.5995]], grad_fn=<So

In [49]:
action_prob_log.shape


torch.Size([34, 2])

In [37]:
loss

tensor(-0.0845, grad_fn=<MeanBackward1>)

In [26]:


for i in range(20):
    cum=0
    state=env.reset()
    done=False
    while not done:
        state=torch.from_numpy(state).unsqueeze(0).float()
        action=net(state).data.numpy().argmax()
        
        #action=env.action_space.sample()

        nextstate,reward,done,_ = env.step(action)
        cum+=reward
        state=nextstate

        env.render()
    print(cum)
    env.close()

136.0
200.0
200.0
200.0
200.0
200.0
200.0
200.0
200.0
200.0
186.0
200.0
200.0
123.0
111.0
200.0
200.0
175.0
171.0
200.0


In [18]:
cum

158.0